<a href="https://colab.research.google.com/github/JHBaek0306/BigData/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_3%EB%B6%80_ML%ED%94%84%EB%A1%9C%EC%84%B8%EC%8A%A42_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B_%EB%B6%84%ED%95%A0%EA%B3%BC_%EB%AA%A8%EB%8D%B8%EA%B2%80%EC%A6%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. 머신러닝 프로세스2: 데이터셋 분할과 모델검증
- 훈련데이터 및 테스트 데이터셋 분할
- 홀드아웃과 교차검증

## 5-1. 특성치(X), 레이블(y) 나누기

In [ ]:
# 데이터셋 불어오기 및 확인
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
data=pd.read_csv('Fvote.csv', encoding='utf-8')
data.head()

,gender_female,gender_male,region_Chungcheung,region_Honam,region_Others,region_Sudo,region_Youngnam,edu,income,age,score_gov,score_progress,score_intention,vote,parties
0,0,1,0,0,0,0,1,3,3,3,2,2,4.0,1,2
1,0,1,0,0,1,0,0,2,3,3,2,4,3.0,0,3
2,0,1,0,1,0,0,0,1,2,4,1,3,2.8,1,4
3,1,0,0,0,0,1,0,2,1,3,5,4,2.6,1,1
4,0,1,0,0,0,1,0,1,2,4,4,3,2.4,1,1


In [ ]:
# 특성변수 데이터셋 나누기
# 방법1: 특성이름으로 데이터셋 나누기
X=data[['gender_female', 'gender_male', 'region_Chungcheung', 'region_Honam',
        'region_Others', 'region_Sudo', 'region_Youngnam', 'edu', 'income',
        'age', 'score_gov', 'score_progress', 'score_intention']]
# 방법2: 특성 위치값으로 데이터셋 나누기
X=data[data.columns[1:14]]
# 방법3: loc 함수로 데이터셋 나누기 (단, 불러올 특성이 연달아 있어야 함)
X=data.loc[:, 'gender_female':'score_intention']

In [ ]:
# 레이블 변수 중 투표여부(vote) 데이터셋 나누기
y=data[["vote"]]

## 5-2. train-test 데이터셋 나누기

In [ ]:
# 훈련데이터와 테스트 데이터 셋 나누기 및 데이터 확인
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, stratify=y, random_state=42)

In [ ]:
# 데이터셋 행렬 구조 확인
print(X_train.shape)
print(X_test.shape)

(158, 13)
(53, 13)


## 5-3. 모델 적용

In [ ]:
# LogisticRegression 알고리즘 적용
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()

### 가. 랜덤없는 교차검증: cross_val_score

In [ ]:
from sklearn.model_selection import cross_val_score
scores=cross_val_score(model, X_train, y_train, cv=5)
print("5개 테스트 셋 정확도:", scores)
print("정확도 평균:", scores.mean())

5개 테스트 셋 정확도: [0.71875    0.6875     0.8125     0.58064516 0.80645161]
정확도 평균: 0.7211693548387096


### 나. 랜덤 있는 교차검증: K-Fold

In [ ]:
from sklearn.model_selection import KFold
kfold=KFold(n_splits=5, shuffle=True, random_state=42)
score=cross_val_score(model, X_train, y_train, cv=kfold)
print("5개 폴드의 정확도:", scores)

5개 폴드의 정확도: [0.71875    0.6875     0.8125     0.58064516 0.80645161]


### 다. 임의분할 교차검증

In [ ]:
from sklearn.model_selection import ShuffleSplit
shuffle_split=ShuffleSplit(test_size=0.5, train_size=0.5, random_state=42)
score=cross_val_score(model, X_train, y_train, cv=shuffle_split)
print("교차검증 정확도:", scores)

교차검증 정확도: [0.71875    0.6875     0.8125     0.58064516 0.80645161]


## 5-4. train-validity-test 분할과 교차검증

In [ ]:
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test=train_test_split(X, y, random_state=1 )
X_train, X_valid, y_train, y_valid=train_test_split(X_train_val, y_train_val, random_state=2 )

In [ ]:
model.fit(X_train, y_train)
scores=cross_val_score(model, X_train, y_train, cv=5)
print("교차검증 정확도:", scores)
print("정확도 평균:", scores.mean())

교차검증 정확도: [0.58333333 0.66666667 0.70833333 0.65217391 0.65217391]
정확도 평균: 0.652536231884058


In [ ]:
model.score(X_valid, y_valid)

0.65

In [ ]:
model.score(X_test, y_test)

0.6981132075471698